In [4]:
import regreg.api as rr
import numpy as np
%load_ext rpy2.ipython

In [5]:
%%R -o X,Y
library(lars)
data(diabetes)
X = diabetes$x
Y = diabetes$y

/home/jb/.virtualenvs/py2/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loaded lars 1.2


  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [6]:
X = np.hstack([X, np.ones((X.shape[0],1))])
#betah = np.dot(np.linalg.pinv(X), Y)
#Yhat = np.dot(X,betah)
#smallest_l2_bound = np.linalg.norm(Y-Yhat)
#null_soln = Y-Y.mean()
#R = smallest_l2_bound / np.linalg.norm(null_soln)

In [7]:
l2 = rr.l2norm.affine(X,-Y,bound=0.65*np.linalg.norm(Y))
l1 = rr.l1norm(X.shape[1], lagrange=1)
l2s = l2.smoothed(rr.identity_quadratic(1.e-12,0,0,0))
problem = rr.simple_problem(l2s, l1)

In [8]:
primal_nesta, dual_nesta = rr.nesta(None, l1, l2)
np.linalg.norm(Y - np.dot(X, primal_nesta)) / np.linalg.norm(Y)

0.65000268709529085

In [9]:
l1_lagrange = rr.l1norm(X.shape[1],lagrange=np.fabs(primal_nesta).sum())
loss = rr.squared_error(X,Y, coef=2)
newsoln = rr.simple_problem(loss, l1_lagrange).solve()
np.linalg.norm(Y - np.dot(X,newsoln)) / np.linalg.norm(Y)

0.31636864339308501

In [10]:
transform, atom = l2.dual
primal_tfocs, dual_tfocs = rr.tfocs(l1, transform, atom)
np.linalg.norm(Y - np.dot(X, primal_tfocs)) / np.linalg.norm(Y)

0.64999986908830232

In [11]:
%timeit primal_tfocs, dual_tfocs = rr.tfocs(l1, transform, atom)

1 loops, best of 3: 1.33 s per loop


In [12]:
%timeit primal_nesta, dual_nesta = rr.nesta(None, l1, l2)

10 loops, best of 3: 173 ms per loop


In [13]:
np.linalg.norm(primal_tfocs - primal_nesta) / (1+np.linalg.norm(primal_nesta))


9.1815670796622612e-06

In [14]:
np.linalg.norm(dual_tfocs - dual_nesta) / (1+np.linalg.norm(dual_nesta))

0.0091156808228944081

## Noiseless case: minimimum L1 norm reconstruction

In [15]:
n, p = 200, 5000
X = np.random.standard_normal((n, p))
l1 = rr.l1norm(p, lagrange=1)
beta = np.zeros(p)
beta[:10] = 10
Y = np.dot(X, beta)

constraint = rr.zero_constraint.affine(X,-Y)
transform, atom = constraint.dual
primal_tfocs, dual_tfocs = rr.tfocs(l1, transform, atom)




/home/jb/code/regreg/regreg/problems/simple.py:358: UserWarning: problem not solved after 100 iterations
  warnings.warn('problem not solved after %d iterations' % max_iters)


In [16]:
np.linalg.norm(Y - np.dot(X, primal_tfocs)) / np.linalg.norm(Y)


5.9576977318511106e-05

In [17]:
primal_tfocs[:20]

array([  9.99932683e+00,   1.00001407e+01,   9.99990989e+00,
         9.99969972e+00,   9.99847969e+00,   9.99856339e+00,
         1.00011371e+01,   1.00002467e+01,   1.00005042e+01,
         1.00003603e+01,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -1.54225134e-06,
         0.00000000e+00,   0.00000000e+00])